In [1]:
import numpy as np
import pydicom
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import nibabel as nib
import skimage
from scipy.stats.distributions import norm
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde
import scipy.integrate as integrate
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.nonparametric.kernel_density import KDEMultivariate
from multiprocessing import Pool
import dill as pickle
from pathos.multiprocessing import ProcessingPool as Pool
warnings.filterwarnings("ignore")

In [2]:
# Data loading
t1_img = nib.load('mni_icbm152_t1_tal_nlin_asym_09a.nii')
t1_data = t1_img.get_data()
t2_img = nib.load('mni_icbm152_t2_tal_nlin_asym_09a.nii')
t2_data = t2_img.get_data()
# Slice of the Nifti image
t1_slice = t1_data[:, :, 94]
t2_slice = t2_data[:, :, 94]

In [3]:
def blur_image(Image_in, epsilon):
    Image = Image_in.copy()
    n = int(Image.size/epsilon)
    for i in range(1,n):
        b = [(Image <= epsilon*i) & (Image >= epsilon*(i-1))]
        Image[b] = 0.5*(epsilon*i + epsilon*(i-1))
    return Image

In [4]:
t1_b = blur_image(t1_slice,1)
t2_b = blur_image(t2_slice,1)

In [5]:
# Flat the vector
t1 = t1_b.flatten()
t2 = t2_b.flatten()

In [7]:
epsilon = 1

In [ ]:
def CR(t1,t2,epsilon):
    def compress(X,Y,epsilon):
        sx = int(np.max(X)/epsilon)
        sy = int(np.max(Y)/epsilon)
        a = np.zeros((sx, sy))
        for i in range(1,sx):
            for j in range(1,sy):
                b = [(X <= epsilon*i) & (X > epsilon*i-1) & (Y <= epsilon*j) & (Y > epsilon*j-1)]
                a[i,j] = np.sum(b)
        return a
    dens = compress(t1,t2,epsilon)
    marginal_x = np.sum(A, axis = 1)
    marginal_y = np.sum(A, axis = 0)
    X = np.arange(epsilon*0.5, int(np.max(t1)/epsilon),epsilon)
    Y = np.arange(epsilon*0.5, int(np.max(t2)/epsilon),epsilon)
    def conditional_variance_x(y):
        py = marginal_y[int(y/epsilon + 0.5)-1]
        if py == 0:
            return 0
        else:
            left = X**2*dens[:,int(y/epsilon + 0.5)-1]/py
            right = X*dens[:,int(y/epsilon + 0.5)-1]/py
            return np.sum(left) - np.sum(right)**2
    def conditional_variance_y(x):
        px = marginal_x[int(x/epsilon + 0.5)-1]
        if px == 0:
            return 0
        else:
            left = Y**2*dens[int(x/epsilon + 0.5)-1,:]/px
            right = Y*dens[int(x/epsilon + 0.5)-1,:]/px
            return np.sum(left) - np.sum(right)**2
    var_x = np.var(X)
    su = 0
    for y in Y:
        su += conditional_variance_x(y)
    left = su/var_x
    var_y = np.var(Y)
    su = 0
    for x in X:
        su += conditional_variance_y(x)
    right su/var_y
    return left + right